https://github.com/bhaskrr/traffic-sign-detection-using-yolov11

In [ ]:
import cv2
from ultralytics import YOLO

# تحميل النموذج المدرب
detector = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt", task="detect")

# فتح كاميرا الويب (0 تعني الكاميرا الافتراضية)
cap = cv2.VideoCapture(0)

# التأكد من فتح الكاميرا
if not cap.isOpened():
    print("خطأ: لم يتم فتح الكاميرا")
    exit()

while True:
    # قراءة إطار من الكاميرا
    ret, frame = cap.read()
    if not ret:
        break

    # إجراء التنبؤ على الإطار الحالي
    detections = detector.predict(frame)

    for detection in detections:
        # الحصول على معرّفات الأصناف
        class_ids = detection.boxes.cls

        # رسم الصناديق على كل كائن مكتشف
        for i, bbox in enumerate(detection.boxes):
            x1, y1, x2, y2 = bbox.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # رسم المستطيل
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            # اسم الفئة
            class_id = int(class_ids[i])
            class_name = detection.names[class_id]

            # عرض اسم الفئة
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # عرض الإطار مع الكائنات المكتشفة
    cv2.imshow("Real-Time Detection", frame)

    # الضغط على "q" للخروج
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# تحرير الموارد
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 898.3ms
Speed: 10.1ms preprocess, 898.3ms inference, 20.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 1391.5ms
Speed: 10.1ms preprocess, 1391.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 666.1ms
Speed: 19.2ms preprocess, 666.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 434.4ms
Speed: 6.1ms preprocess, 434.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 414.1ms
Speed: 13.7ms preprocess, 414.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 403.5ms
Speed: 0.0ms preprocess, 403.5ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 269.9ms
Speed: 0.0ms preprocess, 269.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 383.9ms
Speed: 7.1m

send photo to email 

In [3]:

SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"


In [1]:
import cv2
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from ultralytics import YOLO

# إعدادات الإيميل
SENDER_EMAIL = "ibrahim.mohamed.157248369@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"
RECEIVER_EMAIL = "ibrahim.mohamed.157248369@gmail.com"  # تقدر تغيره لأي بريد

# دالة إرسال الصورة بالإيميل
def send_email_with_image(image_path):
    msg = EmailMessage()
    msg["Subject"] = "🚨 تم اكتشاف إشارة STOP"
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL
    msg.set_content("تم اكتشاف إشارة توقف. الصورة مرفقة.")

    with open(image_path, "rb") as f:
        img_data = f.read()
        msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=os.path.basename(image_path))

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)
        print(f"📧 تم إرسال الصورة إلى {RECEIVER_EMAIL}")
    except Exception as e:
        print(f"❌ خطأ في إرسال الإيميل: {e}")

# تحميل نموذج YOLOv8
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# فتح الكاميرا
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

# إنشاء مجلد لحفظ الصور
save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                # رسم المربع والاسم
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # لو الفئة "stop"، احفظ الصورة وابعته بالإيميل
                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)
                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")
                    send_email_with_image(filename)

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 384.2ms
Speed: 10.1ms preprocess, 384.2ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 546.9ms
Speed: 12.2ms preprocess, 546.9ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 475.0ms
Speed: 10.1ms preprocess, 475.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 515.3ms
Speed: 20.3ms preprocess, 515.3ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 475.3ms
Speed: 9.1ms preprocess, 475.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 331.8ms
Speed: 19.2ms preprocess, 331.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 315.5ms
Speed: 8.1ms preprocess, 315.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 597.5ms
Speed: 5.6ms 

sound effect only 


In [1]:
pip install pygame


Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import os
from datetime import datetime
import pygame  # مكتبة تشغيل الصوت
from ultralytics import YOLO

# تهيئة مكتبة الصوت
pygame.mixer.init()
pygame.mixer.music.load("alert.mp3")  # تأكد إن الملف موجود

# تحميل النموذج
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# فتح الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

# مجلد الصور
save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                # رسم البوكس واسم الفئة
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)

                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")
                    
                    # تشغيل صوت التنبيه
                    if not pygame.mixer.music.get_busy():
                        pygame.mixer.music.play()

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 568.6ms
Speed: 15.0ms preprocess, 568.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 749.1ms
Speed: 43.0ms preprocess, 749.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 958.0ms
Speed: 24.0ms preprocess, 958.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 693.0ms
Speed: 19.0ms preprocess, 693.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 700.0ms
Speed: 44.0ms preprocess, 700.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 525.0ms
Speed: 26.0ms preprocess, 525.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 312.0ms
Speed: 5.0ms preprocess, 312.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 332.0ms
Speed: 8.0ms

sound and email 

In [1]:
SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"


In [ ]:
import cv2
import os
import smtplib
import pygame
from datetime import datetime
from email.message import EmailMessage
from ultralytics import YOLO

# إعدادات الإيميل
SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"
RECEIVER_EMAIL = "ibrahimomoh157@gmail.com"

# تحميل النموذج
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# تهيئة الصوت
pygame.mixer.init()
pygame.mixer.music.load("alert.mp3")

# تشغيل الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

# مجلد الصور
save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

def send_email_with_image(image_path):
    try:
        msg = EmailMessage()
        msg["Subject"] = "🚨 تم اكتشاف إشارة STOP"
        msg["From"] = SENDER_EMAIL
        msg["To"] = RECEIVER_EMAIL
        msg.set_content("تم اكتشاف إشارة توقف. مرفق صورة من الكاميرا.")

        with open(image_path, "rb") as img:
            img_data = img.read()
            msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=os.path.basename(image_path))

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)

        print("📧 تم إرسال الإيميل بنجاح!")
    except Exception as e:
        print("❌ خطأ في إرسال الإيميل:", e)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)

                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")

                    if not pygame.mixer.music.get_busy():
                        pygame.mixer.music.play()

                    send_email_with_image(filename)

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html

0: 480x640 (no detections), 384.8ms
Speed: 9.0ms preprocess, 384.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 459.8ms
Speed: 15.0ms preprocess, 459.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 312.1ms
Speed: 14.0ms preprocess, 312.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 303.9ms
Speed: 5.0ms preprocess, 303.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 438.9ms
Speed: 6.0ms preprocess, 438.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 321.9ms
Speed: 28.0ms preprocess, 321.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 450.0ms
Speed: 10.0ms preprocess, 450.0ms inf

whatsap and email and sound

In [1]:
pip install pywhatkit



     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 145.2 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 140.3 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 204.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished

In [ ]:
import cv2
import os
import smtplib
import pygame
import pywhatkit
from datetime import datetime
from email.message import EmailMessage
from ultralytics import YOLO
import time

# إعدادات الإيميل
SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"
RECEIVER_EMAIL = "ibrahimomoh157@gmail.com"

# رقم الواتساب بصيغة دولية
WHATSAPP_NUMBER = "+201025296838"

# تهيئة مكتبة تشغيل الصوت
pygame.mixer.init()
pygame.mixer.music.load("alert.mp3")  # تأكد إن الملف موجود

# إرسال صورة بالإيميل
def send_email_with_image(image_path):
    msg = EmailMessage()
    msg["Subject"] = "🚨 تم اكتشاف إشارة STOP"
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL
    msg.set_content("تم اكتشاف إشارة توقف. الصورة مرفقة.")

    with open(image_path, "rb") as f:
        img_data = f.read()
        msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=os.path.basename(image_path))

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)
        print(f"📧 تم إرسال الصورة إلى {RECEIVER_EMAIL}")
    except Exception as e:
        print(f"❌ خطأ في إرسال الإيميل: {e}")

# إرسال صورة على واتساب
def send_whatsapp_image(image_path):
    try:
        # تأخير 10 ثواني علشان WhatsApp Web يفتح (يشتغل أول مرة فقط)
        now = datetime.now()
        hour, minute = now.hour, now.minute + 1  # الإرسال بعد دقيقة من الوقت الحالي

        pywhatkit.sendwhats_image(WHATSAPP_NUMBER, image_path, "🚨 تم اكتشاف إشارة STOP", wait_time=10)
        print(f"📲 تم إرسال الصورة على واتساب لـ {WHATSAPP_NUMBER}")
    except Exception as e:
        print(f"❌ خطأ في إرسال واتساب: {e}")

# تحميل النموذج
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# فتح الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)
                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")

                    # تنبيه صوتي
                    if not pygame.mixer.music.get_busy():
                        pygame.mixer.music.play()

                    # إرسال إيميل
                    send_email_with_image(filename)

                    # إرسال واتساب
                    send_whatsapp_image(filename)

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


email and sound and notification

In [ ]:
import cv2
import os
import smtplib
import pygame
import requests
from datetime import datetime
from email.message import EmailMessage
from ultralytics import YOLO

# إعدادات الإيميل
SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"
RECEIVER_EMAIL = "ibrahimomoh157@gmail.com"

# إعدادات Pushover
PUSHOVER_USER_KEY = "uxztq6dxu728prbytfpz8kvz2ozmdo"
PUSHOVER_API_TOKEN = "a6cffbubixz2odrfryy4bsxzux6pks"

# تحميل النموذج
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# تهيئة الصوت
pygame.mixer.init()
pygame.mixer.music.load("alert.mp3")

# تشغيل الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

# مجلد الصور
save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

def send_email_with_image(image_path):
    try:
        msg = EmailMessage()
        msg["Subject"] = "🚨 تم اكتشاف إشارة STOP"
        msg["From"] = SENDER_EMAIL
        msg["To"] = RECEIVER_EMAIL
        msg.set_content("تم اكتشاف إشارة توقف. مرفق صورة من الكاميرا.")

        with open(image_path, "rb") as img:
            img_data = img.read()
            msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=os.path.basename(image_path))

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)

        print("📧 تم إرسال الإيميل بنجاح!")
    except Exception as e:
        print("❌ خطأ في إرسال الإيميل:", e)

def send_push_notification():
    try:
        data = {
            "token": PUSHOVER_API_TOKEN,
            "user": PUSHOVER_USER_KEY,
            "message": "🚨 تم اكتشاف إشارة STOP بواسطة النظام!",
            "title": "تنبيه مروري",
            "priority": 1
        }
        response = requests.post("https://api.pushover.net/1/messages.json", data=data)
        if response.status_code == 200:
            print("📱 تم إرسال إشعار إلى الهاتف!")
        else:
            print("❌ فشل في إرسال الإشعار:", response.text)
    except Exception as e:
        print("❌ خطأ في إرسال الإشعار:", e)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)

                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")

                    if not pygame.mixer.music.get_busy():
                        pygame.mixer.music.play()

                    send_email_with_image(filename)
                    send_push_notification()

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


email and sound and notification with image 

In [3]:
import cv2
import os
import smtplib
import pygame
import requests
from datetime import datetime
from email.message import EmailMessage
from ultralytics import YOLO

# إعدادات الإيميل
SENDER_EMAIL = "ibrahimomoh157@gmail.com"
APP_PASSWORD = "jydkrjribikhvsfa"
RECEIVER_EMAIL = "ibrahimomoh157@gmail.com"

# إعدادات Pushover
PUSHOVER_USER_KEY = "uxztq6dxu728prbytfpz8kvz2ozmdo"
PUSHOVER_API_TOKEN = "a6cffbubixz2odrfryy4bsxzux6pks"

# تحميل النموذج
model = YOLO(r"C:\Users\Ibrahim-sYs\Desktop\outside3\traffic_sign_detector.pt")

# تهيئة الصوت
pygame.mixer.init()
pygame.mixer.music.load("alert.mp3")

# تشغيل الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ لم يتم فتح الكاميرا")
    exit()

# مجلد الصور
save_dir = "detected_stop"
os.makedirs(save_dir, exist_ok=True)

def send_email_with_image(image_path):
    try:
        msg = EmailMessage()
        msg["Subject"] = "🚨 تم اكتشاف إشارة STOP"
        msg["From"] = SENDER_EMAIL
        msg["To"] = RECEIVER_EMAIL
        msg.set_content("تم اكتشاف إشارة توقف. مرفق صورة من الكاميرا.")

        with open(image_path, "rb") as img:
            img_data = img.read()
            msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=os.path.basename(image_path))

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)

        print("📧 تم إرسال الإيميل بنجاح!")
    except Exception as e:
        print("❌ خطأ في إرسال الإيميل:", e)

def send_push_notification_with_image(image_path):
    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": PUSHOVER_API_TOKEN,
        "user": PUSHOVER_USER_KEY,
        "message": "🚨 تم اكتشاف إشارة STOP!",
        "title": "🚦 سيارة رقم 1574 ا ب ج",
        "priority": 1
    }

    with open(image_path, "rb") as img_file:
        files = {"attachment": img_file}
        response = requests.post(url, data=data, files=files)

    if response.status_code == 200:
        print("📲 تم إرسال الإشعار بنجاح!")
    else:
        print(f"❌ فشل في إرسال إشعار الهاتف: {response.status_code} - {response.text}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    for result in results:
        boxes = result.boxes
        names = model.names

        if boxes is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                class_id = int(boxes.cls[i])
                conf = float(boxes.conf[i])
                class_name = names[class_id]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                if class_name.lower() == "stop":
                    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                    filename = os.path.join(save_dir, f"stop_{timestamp}.jpg")
                    cv2.imwrite(filename, frame)

                    print(f"🚨 تم اكتشاف STOP! تم حفظ الصورة: {filename}")

                    if not pygame.mixer.music.get_busy():
                        pygame.mixer.music.play()

                    send_email_with_image(filename)
                    send_push_notification_with_image(filename)

    cv2.imshow("YOLOv8 Real-Time Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html

0: 480x640 (no detections), 352.8ms
Speed: 4.0ms preprocess, 352.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 319.8ms
Speed: 9.0ms preprocess, 319.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 395.9ms
Speed: 5.0ms preprocess, 395.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 538.9ms
Speed: 6.0ms preprocess, 538.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 351.9ms
Speed: 5.0ms preprocess, 351.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 622.8ms
Speed: 5.0ms preprocess, 622.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 321.0ms
Speed: 19.0ms preprocess, 321.0ms infere